In [1]:
%%configure -f 
{
    "conf":
    {
        "spark.app.name":"Bhavesh_Inventory_Metrics",
        "spark.yarn.queue": "default",
        "spark.jars": "/apps/Jars/obelisk-retail-legos-Bhavesh.jar,/apps/Jars/obelisk-retail-usecases-Bhavesh.jar,/apps/Jars/mysql-connector-java-5.1.40.jar",
        "spark.driver.extraJavaOptions": "-Dscala.color",
        "spark.driver.memory": "3g",
        "spark.executor.memory": "3g",
        "spark.executor.instances": "4"
    }
}

In [2]:
import ai.couture.obelisk.commons.Constants._
import ai.couture.obelisk.commons.Constants.STANDARD_COL_NAMES._
import ai.couture.obelisk.commons.Constants._
import ai.couture.obelisk.commons.io._
import ai.couture.obelisk.commons.utils.DateTimeUtil._
import org.apache.spark.sql._
import org.apache.spark.sql.expressions._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._
import spark.implicits._

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1353,application_1685341929318_1929,spark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import ai.couture.obelisk.commons.Constants._
import ai.couture.obelisk.commons.Constants.STANDARD_COL_NAMES._
import ai.couture.obelisk.commons.Constants._
import ai.couture.obelisk.commons.io._
import ai.couture.obelisk.commons.utils.DateTimeUtil._
import org.apache.spark.sql._
import org.apache.spark.sql.expressions._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._
import spark.implicits._


### Local Metrics

In [14]:
def thresholdColumns(columns: Array[String], minThreshold: Double = 0, maxThreshold: Double = Double.PositiveInfinity)(df: DataFrame): DataFrame = {

    var scaledDF = df
    columns.foreach(
    column => {
    scaledDF = scaledDF.withColumn(column,
      when(col(column) > maxThreshold, lit(maxThreshold)).otherwise(col(column)))
      .withColumn(column, when(col(column) < minThreshold, lit(minThreshold)).otherwise(col(column)))
    })
    scaledDF
}

var runDate: String = "2023-04-30"
var baseDir: String = "/data/Archive/bhavesh/InventoryPrediction"
var runDir: String = s"$baseDir/$runDate"
var suffix: String = "colorfamily_pricebucket_styletype_pattern_sleeve_brandname"
var prodLevelSigma = ParquetToDF.getDF(s"$runDir/Statistics12Months/product").select(PRODUCTID, "stddev")
var queryLevelSigma = ParquetToDF.getDF(s"$runDir/Statistics12Months/query").select("query", "stddev", SIMILAR_GROUP_LEVEL, "template")
var productQueryMap = ParquetToDF.getDF(s"$runDir/QueryProductMapping/suffix=$suffix").filter(col("query") =!= "")

def convertLocalToRequiredFormat(dateForWhichPredictionsAreMade: String, basePath: String, level: String, localFile: String, fileType: String = "csv", model: String = "lr"): Unit = {
    
    var actual = ParquetToDF.getDF(s"/data/Archive/bhavesh/InventoryPrediction/actualDataForPredictionPeriod/date=$dateForWhichPredictionsAreMade/sales")
    var fileName: String = if(level == "product"){
       "productLevelPredictions"
    } else{
        "queryAggregatedPredictions"
    }
    def doConversion(brick: String, brickId: String): DataFrame = {
        var localPredictionsPath = s"$basePath/$brick/$localFile"
        var df = if(fileType=="csv"){
            CSVToDF.getDF(localPredictionsPath, inferSchema = true)
        }
        else{
            ParquetToDF.getDF(localPredictionsPath)
        }

        
        df = df
        .withColumn(SIMILAR_GROUP_LEVEL, lit(brickId))
        .withColumn("model", lit(model))
        .withColumn("predicted", col("predictedyQuantity"))
        .select(PRODUCTID, SIMILAR_GROUP_LEVEL, "predicted", "model")
        
        if(level == "product") {
            df.transform(thresholdColumns(Array("predicted")))
            .join(prodLevelSigma, Seq(PRODUCTID))
            .withColumn("lowerBound", col("predicted") - (lit(2.0) * col("stddev")))
            .withColumn("upperBound", col("predicted") + (lit(2.0) * col("stddev")))
            .transform(thresholdColumns(Array("lowerBound")))
        }
        else {
            df.transform(thresholdColumns(Array("predicted")))
            .join(productQueryMap, Seq(SIMILAR_GROUP_LEVEL, PRODUCTID))
            .groupBy("model", "query", SIMILAR_GROUP_LEVEL, "template")
            .agg(sum("predicted") as "predicted")
            .join(queryLevelSigma, Seq(SIMILAR_GROUP_LEVEL, "query", "template"))
            .withColumn("lowerBound", col("predicted") - (lit(2.0) * col("stddev")))
            .withColumn("upperBound", col("predicted") + (lit(2.0) * col("stddev")))
            .transform(thresholdColumns(Array("lowerBound")))
        }
    }
    var menShirts = doConversion("menShirts", "830216013")
    DFToParquet.putDF(s"$basePath/menShirts/$fileName", menShirts, partitionedColumn="model")
    var womenKurtas = doConversion("womenKurtas", "830303011")
    DFToParquet.putDF(s"$basePath/womenKurtas/$fileName", womenKurtas, partitionedColumn="model")
}

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

thresholdColumns: (columns: Array[String], minThreshold: Double, maxThreshold: Double)(df: org.apache.spark.sql.DataFrame)org.apache.spark.sql.DataFrame
runDate: String = 2023-04-30
baseDir: String = /data/Archive/bhavesh/InventoryPrediction
runDir: String = /data/Archive/bhavesh/InventoryPrediction/2023-04-30
suffix: String = colorfamily_pricebucket_styletype_pattern_sleeve_brandname
prodLevelSigma: org.apache.spark.sql.DataFrame = [productid: string, stddev: double]
queryLevelSigma: org.apache.spark.sql.DataFrame = [query: string, stddev: double ... 2 more fields]
productQueryMap: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [query: string, productid: string ... 2 more fields]
convertLocalToRequiredFormat: (dateForWhichPredictionsAreMade: String, basePath: String, level: String, localFile: String, fileType: String, model: String)Unit


In [38]:
def convertLocal(exp: String, date: String, filename: String, format: String, baseDir: String, model: String = "lr"){
    convertLocalToRequiredFormat(date, s"$baseDir/experiments/exp_$exp/predictions/ModelForEachBrickProductLevel", "product", filename, format, model = model)
    convertLocalToRequiredFormat(date, s"$baseDir/experiments/exp_$exp/predictions/ModelForEachBrickProductLevel", "query", filename, format, model = model)
}

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

convertLocal: (exp: String, date: String, filename: String, format: String, baseDir: String, model: String)Unit


In [41]:
convertLocal(
    exp = "neural_network_dropout_lr_0.0005",
    date = "2023-05-01",
    filename = "predictions",
    format = "parquet",
    baseDir = "/data/Archive/bhavesh/InventoryPrediction/2023-04-30",
    model = "neural_network"
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Metrics Show

In [12]:
def metricsShow(exp: String): Unit = {
    var pl = CSVToDF.getDF(s"/data/Archive/bhavesh/InventoryPrediction/2023-04-30/experiments/exp_${exp}/metrics/productLevelModelForEachBrick/productLevel", inferSchema=true).orderBy("model", "similargrouplevel").drop(DATE).persist()
    pl.printSchema

    var show = pl.orderBy("metric").groupBy("model", "metric").pivot("similargrouplevel").agg(first("value")).orderBy("model", "metric").withColumn("final", concat(lit("Men Shirts: "), col("830216013"), lit("\n"), lit("Women Kurtas: "), col("830303011"))).groupBy("model").pivot("metric").agg(first("final") as "final").persist()
    show.orderBy("model").show(false)
    // show.orderBy("model").drop("model").show(false)
    DFToCSV.putDF("/data/Archive/bhavesh/InventoryPrediction/temp/ip/pl_show", show.orderBy("model").coalesce(1))
    pl.unpersist()

    var ql = CSVToDF.getDF(s"/data/Archive/bhavesh/InventoryPrediction/2023-04-30/experiments/exp_${exp}/metrics/productLevelModelForEachBrick/queryLevel", inferSchema=true).orderBy("model", "similargrouplevel").drop(DATE).persist()
    ql.printSchema

    show = ql.orderBy("metric").groupBy("model", "metric").pivot("similargrouplevel").agg(first("value")).orderBy("model", "metric").withColumn("final", concat(lit("Men Shirts: "), col("830216013"), lit("\n"), lit("Women Kurtas: "), col("830303011"))).groupBy("model").pivot("metric").agg(first("final") as "final").persist()
    show.orderBy("model").show(false)
    // show.orderBy("model").drop("model").show(false)
    DFToCSV.putDF("/data/Archive/bhavesh/InventoryPrediction/temp/ip/ql_show", show.orderBy("model").coalesce(1))
    ql.unpersist()
}

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

metricsShow: (exp: String)Unit


In [28]:
var ql = CSVToDF.getDF("/data/Archive/bhavesh/InventoryPrediction/2023-04-30/experiments/combined/nn_crostonsba/metrics/date=2023-05-01").orderBy("similargrouplevel").persist()
ql.printSchema
var show = ql.groupBy("metric").pivot("similargrouplevel").agg(first("value")).orderBy("metric").withColumn("final", concat(lit("Men Shirts: "), col("830216013"), lit("\n"), lit("Women Kurtas: "), col("830303011"))).withColumn("model", lit("combined")).groupBy("model").pivot("metric").agg(first("final") as "final").persist()
show.show()
DFToCSV.putDF("/data/Archive/bhavesh/InventoryPrediction/temp/ip/ql_show", show.coalesce(1))
ql.unpersist()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

ql: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [metric: string, value: string ... 1 more field]
root
 |-- metric: string (nullable = true)
 |-- value: string (nullable = true)
 |-- similargrouplevel: integer (nullable = true)

show: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [model: string, Bias: string ... 7 more fields]
+--------+--------------------+--------------------+---------------------------+-------------------------------------------------+------------------------------------------------+------------------------------------------------+------------------------------------------------+------------------------------------------------+
|   model|                Bias|            Hit Rate|Mean Absolute Percent Error|Root Mean Square Error for 100.0 percentile error|Root Mean Square Error for 25.0 percentile error|Root Mean Square Error for 50.0 percentile error|Root Mean Square Error for 75.0 percentile error|Root Mean Square Error for 90.0 percentile e

In [13]:
val exp_code = "1"
metricsShow(exp_code)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

exp_code: String = 1
root
 |-- metric: string (nullable = true)
 |-- value: double (nullable = true)
 |-- model: string (nullable = true)
 |-- similargrouplevel: integer (nullable = true)

+-----+----------------------------------------------------------------+---------------------------------------------------------------+---------------------------------------------------------------+--------------------------------------------------------------+---------------------------------------------------------------+--------------------------------------------------------------+--------------------------------------------------------------+---------------------------------------------------------------+
|model|Bias                                                            |Hit Rate                                                       |Mean Absolute Percent Error                                    |Root Mean Square Error for 100.0 percentile error             |Root Mean Square Error for 25.